## Import Modules

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import urllib.request
import os
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Load TripAdvisior data

In [3]:

trip_df = pd.read_excel("D:/TripAdvisor/TripAdvisor.xlsx",sheet_name = "in")


## Extract Target Column

In [4]:
trip_df['Contributions'].fillna('',inplace = True)
trip_df['Helpulness'] = trip_df['Contributions'].apply(lambda x:re.search('(\d+)(\s*)helpful', x, re.IGNORECASE))
trip_df['Helpulness'] = trip_df['Helpulness'].apply(lambda x:int(x.group(1)) if x is not None else 0)
trip_df.tail()
# Convert DataFrame to Excel
#trip_df.to_excel('trip_data.xlsx', index=False)

,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness
13723,David R,"Narberth, United States;1 contribution",['Very luxurious'],Beautiful hotel with outstanding service & dra...,Date of stay: June 2015,CONRAD NEW YORK DOWNTOWN - Updated 2023 Prices...,0
13724,Kate J,"London, United Kingdom;7 contributions;11 help...",['Very Impressed - will return'],I booked a standard room quite late in the day...,Date of stay: May 2015,CONRAD NEW YORK DOWNTOWN - Updated 2023 Prices...,11
13725,travellingangie,"Baltimore, Maryland;467 contributions;168 help...",['excellent stay'],"this was an excellent hotel, located maybe 4 b...",Date of stay: June 2015,CONRAD NEW YORK DOWNTOWN - Updated 2023 Prices...,168
13726,Trend_Dan,"Singapore, Singapore;63 contributions;60 helpf...",['A big disappointment'],"Having to stay in Conrad Maldives, Tokyo, Seou...",Date of stay: June 2015,CONRAD NEW YORK DOWNTOWN - Updated 2023 Prices...,60
13727,DiveCaribbean5,"Washington, DC;22 contributions;10 helpful votes",['Surprisingly baby friendly hotel'],We've stayed at this hotel a couple of times w...,Date of stay: March 2015,CONRAD NEW YORK DOWNTOWN - Updated 2023 Prices...,10


## Generating label column

In [5]:

trip_df['Label'] = trip_df['Helpulness'].apply(lambda x: 1 if x >= 1 else 0)
trip_df.to_excel('trip_bert.xlsx', index=False)

In [6]:
trip_df.head()

,User_Name,Contributions,Title,Review,Date_of_Stay,Heading,Helpulness,Label
0,T5329NXmaryh,"Louisville, Kentucky;3 contributions",['Shantel & Luigi welcomed us with smiles and ...,Shantel & Luigi and all front desk staff are e...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,0
1,jcmtl1965,"Montreal, Canada;8938 contributions;8938;100 h...",['Nice stay for my return to New York'],I hadn't been to New York in 22 months (which ...,Date of stay: August 2021,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,100,1
2,Venture02710567543,1 contribution,['Perfect stay with perfect view'],The convenience of checking in with Nakita Wan...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,0
3,Leigh H,1 contribution,['Service with a smile'],"The front desk staff at our check-in, Nakita, ...",Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,0
4,GrandTour48028656883,1 contribution,['Highly recommend!'],Everyone at this hotel was great! Jaquanna hel...,Date of stay: June 2023,HILTON GARDEN INN NYC FINANCIAL CENTER / MANHA...,0,0


## Subsetting and Splitting the data

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd



# Assuming you already have the 'trip_df' DataFrame
trip_df['Review'] = trip_df['Review'].fillna('')
trip_df['Review'] = trip_df['Review'].astype(str)
trip_df['Review'] = trip_df['Review'].str.lower()


# Select only the first 600 rows from the 'trip_df' DataFrame
trip_df_subset = trip_df.iloc[:600, :]


# Split the DataFrame into training and validation sets
trip_df_train, trip_df_val = train_test_split(trip_df_subset, test_size=0.2, random_state=42)

# Print the resulting DataFrames
print("Training Set:")
print(trip_df_train.shape)

print("Validation Set:")
print(trip_df_val.shape)


Training Set:
(480, 8)
Validation Set:
(120, 8)


In [8]:
#from sklearn.model_selection import train_test_split
#import pandas as pd

# Read the Excel file into a DataFrame
#df = pd.read_excel('/content/data.xlsx')
#trip_df['Review'] = trip_df['Review'].fillna('')
#trip_df['Review'] = trip_df['Review'].astype(str)
#trip_df['Review'] = trip_df['Review'].str.lower()

# Split the DataFrame into training and validation sets
#trip_df_train, trip_df_val = train_test_split(trip_df, test_size=0.08, random_state=42)

# Print the resulting DataFrames
#print("Training Set:")
#df_train.head()

#print("Validation Set:")
#trip_df_val.head()
#trip_df_val.shape

## Initiate Tokenizer and bert model

In [9]:
!pip install transformers

import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

cpu


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Tokenize the reviews

In [10]:
tokenized_train = tokenizer(trip_df_train["Review"].values.tolist(), padding = True, truncation = True, return_tensors="pt")
tokenized_val = tokenizer(trip_df_val["Review"].values.tolist() , padding = True, truncation = True,  return_tensors="pt")

print(tokenized_train.keys())
print(tokenized_train.items())
#move on device (GPU)
tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
tokenized_val = {k:torch.tensor(v).to(device) for k,v in tokenized_val.items()}
# Print the shapes of the tensors in the 'tokenized_train' dictionary
for key, tensor in tokenized_val.items():
    print(f"{key} shape: {tensor.shape}")
for key, tensor in tokenized_train.items():
    print(f"{key} shape: {tensor.shape}")

dict_keys(['input_ids', 'attention_mask'])
dict_items([('input_ids', tensor([[ 101, 1045, 4370,  ...,    0,    0,    0],
        [ 101, 2200, 6625,  ...,    0,    0,    0],
        [ 101, 3309, 2001,  ...,    0,    0,    0],
        ...,
        [ 101, 2034, 1010,  ...,    0,    0,    0],
        [ 101, 1045, 2001,  ...,    0,    0,    0],
        [ 101, 2023, 1037,  ...,    0,    0,    0]])), ('attention_mask', tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]))])
input_ids shape: torch.Size([120, 205])
attention_mask shape: torch.Size([120, 205])
input_ids shape: torch.Size([480, 204])
attention_mask shape: torch.Size([480, 204])


C:\Users\admin\AppData\Local\Temp\ipykernel_20520\1862095452.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
C:\Users\admin\AppData\Local\Temp\ipykernel_20520\1862095452.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_val = {k:torch.tensor(v).to(device) for k,v in tokenized_val.items()}


## Genearating Hidden Layers and Separating CLS

In [11]:
with torch.no_grad():
  hidden_train = model(**tokenized_train) #dim : [batch_size(nr_sentences), tokens, emb_dim]
  hidden_val = model(**tokenized_val)

#get only the [CLS] hidden states
cls_train = hidden_train.last_hidden_state[:,0,:]
cls_val = hidden_val.last_hidden_state[:,0,:]

In [12]:
print(cls_train)
print(cls_val)

tensor([[-0.0345, -0.1487,  0.2817,  ...,  0.0992,  0.3654,  0.2890],
        [-0.0564, -0.2833,  0.3037,  ..., -0.4367,  0.4253,  0.1608],
        [ 0.0373, -0.2357,  0.1162,  ..., -0.3637,  0.5566,  0.0560],
        ...,
        [-0.0259, -0.0666,  0.1747,  ..., -0.0581,  0.4632,  0.3592],
        [ 0.1399, -0.1187,  0.0367,  ...,  0.0041,  0.3943,  0.2891],
        [ 0.0226, -0.0072,  0.3378,  ..., -0.1710,  0.6015,  0.2064]])
tensor([[ 0.0981, -0.1501,  0.2193,  ..., -0.0686,  0.3885,  0.2730],
        [-0.0148,  0.0284,  0.1011,  ...,  0.0191,  0.5658,  0.3365],
        [ 0.1234, -0.1156, -0.0148,  ..., -0.0413,  0.2973,  0.1023],
        ...,
        [ 0.1286,  0.0722,  0.1851,  ..., -0.0448,  0.4759,  0.1226],
        [ 0.3463, -0.2160, -0.0551,  ..., -0.0105,  0.5235,  0.4116],
        [ 0.0613, -0.1977,  0.0385,  ..., -0.0424,  0.4499,  0.1971]])


### Random Forest Classifier

In [13]:
x_train = cls_train.to("cpu")
y_train=trip_df_train["Label"]

x_val = cls_val.to("cpu")
y_val = trip_df_val["Label"]

print(x_train.shape,y_train.shape,x_val.shape,y_val.shape)


from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier()
rf.fit(x_train,y_train)
rf.score(x_val,y_val)


#cross validation ?

torch.Size([480, 768]) (480,) torch.Size([120, 768]) (120,)


0.7583333333333333

## Random Forest with CV

In [14]:
import numpy as np
from sklearn.model_selection import cross_val_score

# Combine the training and validation data
x_train_val = np.vstack((x_train, x_val))
y_train_val = np.concatenate((y_train, y_val))

# Create a new RandomForestClassifier instance
rf = RandomForestClassifier()

# Perform 5-fold cross-validation (you can choose a different number of folds)
num_folds = 5
scores = cross_val_score(rf, x_train_val, y_train_val, cv=num_folds)

# Print the cross-validation scores and the average score
print("Cross-Validation Scores:", scores)
print("Mean Cross-Validation Score:", scores.mean())


Cross-Validation Scores: [0.725      0.725      0.66666667 0.70833333 0.725     ]
Mean Cross-Validation Score: 0.7100000000000001


In [15]:
!pip install tensorflow
!pip install transformers


### Import the required libraries:

In [16]:
import tensorflow as tf
from sklearn.model_selection import KFold
from transformers import TFDistilBertForSequenceClassification

### Load the pre-trained DistilBERT model:


In [17]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [18]:
y_train=trip_df_train["Label"]
y_val = trip_df_val["Label"]

In [19]:
# Convert the labels to numpy arrays if they are not already
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

 # This code converts the labels (target values) y_train and y_val into NumPy arrays. This is done to ensure compatibility with TensorFlow datasets.

In [20]:
import torch

# Assuming you have the following variables:
# 'tokenized_train' and 'tokenized_val' are dictionaries containing 'input_ids' and 'attention_mask' tensors.

# Choose a common maximum sequence length that accommodates both datasets
max_seq_length = 205

# Pad or truncate the sequences in 'input_ids' and 'attention_mask' tensors to have the same length
def pad_sequences(tensor, max_length):
    num_pad = max_length - tensor.size(1)
    if num_pad > 0:
        pad_tensor = torch.zeros((tensor.size(0), num_pad), dtype=tensor.dtype, device=tensor.device)
        tensor = torch.cat([tensor, pad_tensor], dim=1)
    return tensor

tokenized_train['input_ids'] = pad_sequences(tokenized_train['input_ids'], max_seq_length)
tokenized_train['attention_mask'] = pad_sequences(tokenized_train['attention_mask'], max_seq_length)

tokenized_val['input_ids'] = pad_sequences(tokenized_val['input_ids'], max_seq_length)
tokenized_val['attention_mask'] = pad_sequences(tokenized_val['attention_mask'], max_seq_length)


In [21]:
import numpy as np
import torch

# Assuming you have a dictionary called 'tokenized_train'
# Access the 'input_ids' and 'attention_mask' tensors
input_ids_tensor = tokenized_train['input_ids']
attention_mask_tensor = tokenized_train['attention_mask']

# Move the tensors to CPU and convert them to NumPy arrays
input_ids_np = input_ids_tensor.cpu().numpy()
attention_mask_np = attention_mask_tensor.cpu().numpy()

# Assuming you have 'tokenized_val' as a dictionary with 'input_ids' and 'attention_mask' tensors
# Access the 'input_ids' and 'attention_mask' tensors
input_ids_tensor_val = tokenized_val['input_ids']
attention_mask_tensor_val = tokenized_val['attention_mask']

# Move the tensors to CPU and convert them to NumPy arrays
input_ids_val = input_ids_tensor_val.cpu().numpy()
attention_mask_val = attention_mask_tensor_val.cpu().numpy()


In [22]:
import tensorflow as tf

# Assuming you have 'input_ids_np' and 'attention_mask_np' as NumPy arrays
# and 'y_train' and 'y_val' as NumPy arrays for the labels

# Create the train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': input_ids_np, 'attention_mask': attention_mask_np},
    y_train
)).shuffle(len(input_ids_np)).batch(32)

# Assuming you have 'input_ids_val' and 'attention_mask_val' as NumPy arrays for validation data
# Create the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((
    {'input_ids': input_ids_val, 'attention_mask': attention_mask_val},
    y_val
)).batch(64)


In [23]:

# Define the optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model with the optimizer and loss function
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=2, validation_data=val_dataset)

# Evaluate the model
loss, accuracy = model.evaluate(val_dataset)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)




Epoch 1/2
15/15 [==============================] - 262s 17s/step - loss: 0.6258 - accuracy: 0.6854 - val_loss: 0.5898 - val_accuracy: 0.7167
Epoch 2/2
2/2 [==============================] - 21s 10s/step - loss: 0.5834 - accuracy: 0.7167
Validation Loss: 0.5834433436393738
Validation Accuracy: 0.7166666388511658


In [24]:
print("Shape of input_ids_np:", input_ids_np.shape)
print("Shape of input_ids_val:", input_ids_val.shape)


Shape of input_ids_np: (480, 205)
Shape of input_ids_val: (120, 205)


## Cross Validation

In [25]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold

# Assuming you have 'input_ids_np', 'attention_mask_np', 'y_train', and 'y_val' as NumPy arrays

# Combine the training and validation data
x_train_val = np.concatenate((input_ids_np, input_ids_val), axis=0)
y_train_val = np.concatenate((y_train, y_val), axis=0)

# Define the number of folds for cross-validation
num_folds = 3

# Initialize KFold cross-validator
kfold = KFold(n_splits=num_folds, shuffle=True)
validation_accuracies = []
# Perform cross-validation
fold = 1
for train_idx, val_idx in kfold.split(x_train_val):
    print(f"Training Fold {fold}...")

    # Get the training and validation data for this fold
    x_train_fold, x_val_fold = x_train_val[train_idx], x_train_val[val_idx]
    y_train_fold, y_val_fold = y_train_val[train_idx], y_train_val[val_idx]

    # Create the train dataset for this fold
    train_dataset = tf.data.Dataset.from_tensor_slices((
        {'input_ids': x_train_fold},
        y_train_fold
    )).shuffle(len(x_train_fold)).batch(32)

    # Create the validation dataset for this fold
    val_dataset = tf.data.Dataset.from_tensor_slices((
        {'input_ids': x_val_fold},
        y_val_fold
    )).batch(64)


    # Train the model on the current fold
    model.fit(train_dataset, epochs=2, validation_data=val_dataset)

    # Evaluate the model on the validation set of this fold
    loss, accuracy = model.evaluate(val_dataset)
    print(f"Fold {fold} Validation Loss: {loss}, Validation Accuracy: {accuracy}")
    # Append the validation accuracy to the list
    validation_accuracies.append(accuracy)


    fold += 1
# Calculate the overall accuracy (average of validation accuracies)
overall_accuracy = np.mean(validation_accuracies)
print("Overall Accuracy:", overall_accuracy)


Training Fold 1...
Epoch 1/2


13/13 [==============================] - 224s 17s/step - loss: 0.5876 - accuracy: 0.6925 - val_loss: 0.5464 - val_accuracy: 0.7450
Epoch 2/2
4/4 [==============================] - 31s 7s/step - loss: 0.5263 - accuracy: 0.7550
Fold 1 Validation Loss: 0.5262860655784607, Validation Accuracy: 0.7549999952316284
Training Fold 2...
Epoch 1/2
13/13 [==============================] - 234s 18s/step - loss: 0.5252 - accuracy: 0.7325 - val_loss: 0.4965 - val_accuracy: 0.7900
Epoch 2/2
4/4 [==============================] - 34s 8s/step - loss: 0.4683 - accuracy: 0.8200
Fold 2 Validation Loss: 0.46834519505500793, Validation Accuracy: 0.8199999928474426
Training Fold 3...
Epoch 1/2
13/13 [==============================] - 220s 17s/step - loss: 0.4499 - accuracy: 0.7975 - val_loss: 0.3295 - val_accuracy: 0.8800
Epoch 2/2
4/4 [==============================] - 31s 7s/step - loss: 0.2871 - accuracy: 0.9050
Fold 3 Validation Loss: 0.2870711386203766, Validation Accuracy: 0.9049999713897705
Overall Acc